In [48]:
import pandas as pd
import numpy as np

df = pd.read_csv('sensex.csv')
print(df.head())

        Price        Close         High          Low         Open  Volume
0        Date          NaN          NaN          NaN          NaN     NaN
1  1997-07-01  4300.859863  4301.770020  4247.660156  4263.109863     0.0
2  1997-07-02  4333.899902  4395.310059  4295.399902  4302.959961     0.0
3  1997-07-03  4323.459961  4393.290039  4299.970215  4335.790039     0.0
4  1997-07-04  4323.819824  4347.589844  4300.580078  4332.700195     0.0


In [49]:
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')
    df.set_index('Date', inplace=True)

print("\nDataFrame after processing the Date column:")
print(df.head())


DataFrame after processing the Date column:
        Price        Close         High          Low         Open  Volume
0        Date          NaN          NaN          NaN          NaN     NaN
1  1997-07-01  4300.859863  4301.770020  4247.660156  4263.109863     0.0
2  1997-07-02  4333.899902  4395.310059  4295.399902  4302.959961     0.0
3  1997-07-03  4323.459961  4393.290039  4299.970215  4335.790039     0.0
4  1997-07-04  4323.819824  4347.589844  4300.580078  4332.700195     0.0


In [50]:
df['Daily_Return'] = df['Close'].pct_change() * 100

# Define a threshold for a daily crash (e.g., drop more than 5%)
crash_threshold_daily = -5
df['Crash_Daily'] = df['Daily_Return'] <= crash_threshold_daily

In [51]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=df.index, y=df['Close'], mode='lines', name='Closing Price'))

crash_days = df.index[df['Crash_Daily']]
crash_closes = df['Close'][df['Crash_Daily']]

fig.add_trace(go.Scatter(x=crash_days, y=crash_closes, mode='markers',
                          marker=dict(color='red'),
                          name=f'Daily Crash (<= {crash_threshold_daily}%)'))

fig.update_layout(
    title='Sensex Closing Price with Daily Crashes Highlighted',
    xaxis_title='Date',
    yaxis_title='Sensex Close',
    xaxis_rangeslider_visible=True,  
    template="plotly_white" 

)

fig.show()

In [52]:
# Calculate cumulative max and drawdown (% drop from the cumulative max)
df['Cumulative_Max'] = df['Close'].cummax()
df['Drawdown'] = (df['Close'] - df['Cumulative_Max']) / df['Cumulative_Max'] * 100

In [53]:
drawdown_threshold = -20
fig = go.Figure()

fig.add_trace(go.Scatter(x=df.index, y=df['Drawdown'], mode='lines', name='Drawdown (%)'))

fig.add_hline(y=drawdown_threshold, line_dash='dash', line_color='red',
              annotation_text=f'Drawdown Threshold ({drawdown_threshold}%)',
              annotation_position='bottom right')

fig.update_layout(
    title='Market Drawdown Over Time',
    xaxis_title='Date',
    yaxis_title='Drawdown (%)',
    xaxis_rangeslider_visible=True,  
    template="plotly_white"
)

fig.show()

In [54]:
crash_drawdowns = df[df['Drawdown'] <= drawdown_threshold]
print("Dates where drawdown exceeded threshold:")
print(crash_drawdowns[['Close', 'Cumulative_Max', 'Drawdown']].dropna().head(10))

Dates where drawdown exceeded threshold:
          Close  Cumulative_Max   Drawdown
90  3633.179932      4548.02002 -20.115129
91  3554.100098      4548.02002 -21.853904
92  3569.770020      4548.02002 -21.509360
93  3578.100098      4548.02002 -21.326202
94  3518.899902      4548.02002 -22.627871
95  3454.649902      4548.02002 -24.040574
96  3466.860107      4548.02002 -23.772101
97  3523.439941      4548.02002 -22.528047
98  3403.070068      4548.02002 -25.174690
99  3479.889893      4548.02002 -23.485607


In [55]:
import pandas as pd

# Example: ensure both are datetime
date = pd.to_datetime(date)
last_date = pd.to_datetime(current_cluster[-1])

if (date - last_date).days <= 3:
    current_cluster.append(date)
else:
    clusters.append(current_cluster)
    current_cluster = [date]


In [57]:
# Get dates where drawdown exceeds threshold
crash_dates = df.index[df['Drawdown'] <= drawdown_threshold]
clusters = []
current_cluster = []

for date in crash_dates:
    if not current_cluster:
        current_cluster.append(date)
    else:
        # Ensure both dates are datetime objects
        current_date = pd.to_datetime(df.loc[date, 'Price'])
        last_date = pd.to_datetime(df.loc[current_cluster[-1], 'Price'])
        date_diff = (current_date - last_date).days
        
        if date_diff <= 3:
            current_cluster.append(date)
        else:
            clusters.append(current_cluster)
            current_cluster = [date]

In [68]:
print(df.columns)
print(df.index[:5])


Index(['Price', 'Close', 'High', 'Low', 'Open', 'Volume', 'Daily_Return',
       'Crash_Daily', 'Cumulative_Max', 'Drawdown'],
      dtype='object')
RangeIndex(start=0, stop=5, step=1)


In [72]:
# Ensure we create a Date column safely (avoid NameError when original_date_list isn't available)
# Prefer existing 'Date' column; if not present, try to parse the 'Price' column which contains date strings.
if 'Date' not in df.columns or df['Date'].isnull().all():
	df['Date'] = pd.to_datetime(df.get('Price'), errors='coerce')

# If still no valid dates, provide a clear error
if df['Date'].isnull().all():
	raise ValueError("No valid dates found in df['Date'] or df['Price']. Provide a valid 'original_date_list' or ensure one of those columns has date strings.")

# Finally set the index to Date
df.set_index('Date', inplace=True)


C:\Users\Acer\AppData\Local\Temp\ipykernel_37512\1296026367.py:4: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



In [74]:
import pandas as pd
import plotly.graph_objects as go

# 🔹 Create a synthetic sequential Date column if not present
if 'Date' not in df.columns:
    df['Date'] = pd.date_range(start='2020-01-01', periods=len(df), freq='D')

# Ensure 'Date' is datetime
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

if clusters:
    # Use the first cluster for demonstration
    cluster_start_idx = clusters[0][0]
    cluster_end_idx = clusters[0][-1]

    # ✅ Use .iloc to get rows by position instead of index label
    cluster_start_dt = df.iloc[cluster_start_idx]['Date']
    cluster_end_dt = df.iloc[cluster_end_idx]['Date']

    start_date_str = cluster_start_dt.date()
    end_date_str = cluster_end_dt.date()

    print(f"\nZooming into crash cluster from {start_date_str} to {end_date_str}")

    # ✅ Define zoom range (30 days before and after)
    zoom_start = cluster_start_dt - pd.Timedelta(days=30)
    zoom_end = cluster_end_dt + pd.Timedelta(days=30)

    # ✅ Filter the DataFrame by date range
    zoom_df = df[(df['Date'] >= zoom_start) & (df['Date'] <= zoom_end)]

    # === Plot 1: Closing Price ===
    fig_close = go.Figure()
    fig_close.add_trace(go.Scatter(
        x=zoom_df['Date'],
        y=zoom_df['Close'],
        mode='lines',
        name='Closing Price'
    ))
    fig_close.add_vrect(
        x0=cluster_start_dt, x1=cluster_end_dt,
        fillcolor='red', opacity=0.3, layer='below', line_width=0
    )
    fig_close.update_layout(
        title=f'Sensex Closing Price from {zoom_start.date()} to {zoom_end.date()}',
        xaxis_title='Date',
        yaxis_title='Sensex Close',
        xaxis_rangeslider_visible=True,
        template="plotly_white"
    )
    fig_close.show()

    # === Plot 2: Daily Returns ===
    fig_returns = go.Figure()
    fig_returns.add_trace(go.Scatter(
        x=zoom_df['Date'],
        y=zoom_df['Daily_Return'],
        mode='lines',
        name='Daily Return (%)'
    ))
    fig_returns.add_hline(y=0, line_color='black', line_width=0.8)
    fig_returns.add_vrect(
        x0=cluster_start_dt, x1=cluster_end_dt,
        fillcolor='red', opacity=0.3, layer='below', line_width=0
    )
    fig_returns.update_layout(
        title=f'Sensex Daily Returns from {zoom_start.date()} to {zoom_end.date()}',
        xaxis_title='Date',
        yaxis_title='Daily Return (%)',
        xaxis_rangeslider_visible=True,
        template="plotly_white"
    )
    fig_returns.show()

    # === Plot 3: Drawdown ===
    fig_drawdown = go.Figure()
    fig_drawdown.add_trace(go.Scatter(
        x=zoom_df['Date'],
        y=zoom_df['Drawdown'],
        mode='lines',
        name='Drawdown (%)'
    ))
    fig_drawdown.add_hline(
        y=drawdown_threshold,
        line_dash='dash',
        line_color='red',
        annotation_text=f'Drawdown Threshold ({drawdown_threshold}%)',
        annotation_position='bottom right'
    )
    fig_drawdown.add_vrect(
        x0=cluster_start_dt, x1=cluster_end_dt,
        fillcolor='red', opacity=0.3, layer='below', line_width=0
    )
    fig_drawdown.update_layout(
        title=f'Sensex Drawdown from {zoom_start.date()} to {zoom_end.date()}',
        xaxis_title='Date',
        yaxis_title='Drawdown (%)',
        xaxis_rangeslider_visible=True,
        template="plotly_white"
    )
    fig_drawdown.show()

else:
    print("No crash clusters identified based on the drawdown threshold.")



Zooming into crash cluster from 2020-03-31 to 2020-05-01


In [76]:
import pandas as pd
import plotly.graph_objects as go

# 🔹 Step 1: Ensure Date column exists
if 'Date' not in df.columns:
    # If no real dates, generate synthetic daily dates
    df['Date'] = pd.date_range(start='1997-01-01', periods=len(df), freq='D')

# 🔹 Step 2: Convert to datetime
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# 🔹 Step 3: Set Date as index (important for slicing with timestamps)
df.set_index('Date', inplace=True)

# === Define the crash clusters ===
cluster1_start = pd.to_datetime("1997-11-12")
cluster1_end   = pd.to_datetime("1997-12-26")

cluster49_start = pd.to_datetime("2008-08-18")
cluster49_end   = pd.to_datetime("2009-01-23")

cluster79_start = pd.to_datetime("2020-03-16")
cluster79_end   = pd.to_datetime("2020-04-03")

# === Plotting functions ===
def plot_crash_period(cluster_start, cluster_end, title_suffix=""):
    zoom_start = cluster_start - pd.Timedelta(days=30)
    zoom_end   = cluster_end + pd.Timedelta(days=30)

    # ✅ Slice using DatetimeIndex
    zoom_df = df.loc[zoom_start:zoom_end]

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=zoom_df.index,
        y=zoom_df['Close'],
        mode='lines',
        name='Closing Price'
    ))
    fig.add_vrect(
        x0=cluster_start,
        x1=cluster_end,
        fillcolor='red',
        opacity=0.3,
        layer='below',
        line_width=0,
        name='Crash Period'
    )
    fig.update_layout(
        title=f'Sensex Closing Price {title_suffix}<br>({zoom_start.date()} to {zoom_end.date()})',
        xaxis_title='Date',
        yaxis_title='Sensex Close',
        xaxis_rangeslider_visible=True,
        template="plotly_white"
    )
    fig.show()

def plot_daily_returns(cluster_start, cluster_end, title_suffix=""):
    zoom_start = cluster_start - pd.Timedelta(days=30)
    zoom_end   = cluster_end + pd.Timedelta(days=30)

    zoom_df = df.loc[zoom_start:zoom_end]

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=zoom_df.index,
        y=zoom_df['Daily_Return'],
        mode='lines',
        name='Daily Return (%)'
    ))
    fig.add_hline(y=0, line_color='black', line_width=0.8)
    fig.add_vrect(
        x0=cluster_start,
        x1=cluster_end,
        fillcolor='red',
        opacity=0.3,
        layer='below',
        line_width=0,
        name='Crash Period'
    )
    fig.update_layout(
        title=f'Sensex Daily Returns {title_suffix}<br>({zoom_start.date()} to {zoom_end.date()})',
        xaxis_title='Date',
        yaxis_title='Daily Return (%)',
        xaxis_rangeslider_visible=True,
        template="plotly_white"
    )
    fig.show()

# === Step 4: Plot all crash periods ===
plot_crash_period(cluster1_start, cluster1_end, "1997 Crash")
plot_crash_period(cluster49_start, cluster49_end, "2008-2009 Crash")
plot_crash_period(cluster79_start, cluster79_end, "2020 Crash")

plot_daily_returns(cluster1_start, cluster1_end, "1997 Crash")
plot_daily_returns(cluster49_start, cluster49_end, "2008-2009 Crash")
plot_daily_returns(cluster79_start, cluster79_end, "2020 Crash")


In [77]:
np.random.seed(42)

dates_2025 = pd.bdate_range(start="2025-01-01", periods=250)

daily_returns = np.zeros(250)
daily_returns[:150] = np.random.normal(loc=0.0005, scale=0.01, size=150)
daily_returns[150:200] = np.random.normal(loc=-0.008, scale=0.025, size=50)
daily_returns[200:] = np.random.normal(loc=0.0005, scale=0.01, size=50)

prices = [30000]
for ret in daily_returns:
    prices.append(prices[-1]*(1+ret))
prices = prices[1:]

df_2025 = pd.DataFrame({
    'Date': dates_2025,
    'Close': prices,
    'Daily_Return': daily_returns * 100 
})
df_2025.set_index('Date', inplace=True)

In [78]:
df_2025['Rolling_Mean_Return'] = df_2025['Daily_Return'].rolling(window=10).mean()
df_2025['Rolling_Volatility'] = df_2025['Daily_Return'].rolling(window=10).std()

warning_condition = (df_2025['Rolling_Mean_Return'] < -0.5) & (df_2025['Rolling_Volatility'] > 2)
df_2025['Warning'] = warning_condition

warnings_df = df_2025[df_2025['Warning']]
print("Early Warning Signals for 2025:")
print(warnings_df[['Close', 'Daily_Return', 'Rolling_Mean_Return', 'Rolling_Volatility']].head(15))

Early Warning Signals for 2025:
                   Close  Daily_Return  Rolling_Mean_Return  \
Date                                                          
2025-08-13  26293.754799     -3.236704            -0.723106   
2025-08-14  26600.790000      1.167712            -0.612947   
2025-08-29  25310.851435     -0.992754            -0.676806   
2025-09-02  25296.816487     -0.108273            -0.558018   
2025-09-04  25420.955891     -0.767495            -0.656682   
2025-09-23  23735.864103     -4.587118            -1.372651   
2025-09-25  23593.214467      1.340997            -1.212308   
2025-09-29  22609.672636     -3.914347            -1.323114   
2025-09-30  22526.684356     -0.367048            -1.223953   

            Rolling_Volatility  
Date                            
2025-08-13            2.316762  
2025-08-14            2.383681  
2025-08-29            2.373521  
2025-09-02            2.146429  
2025-09-04            2.130037  
2025-09-23            2.003797  
2025-09-25

In [79]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_2025.index, y=df_2025['Close'], mode='lines', name='Closing Price'))

warning_dates = df_2025.index[df_2025['Warning']]
warning_closes = df_2025['Close'][df_2025['Warning']]

fig.add_trace(go.Scatter(x=warning_dates, y=warning_closes, mode='markers',
                          marker=dict(color='red'),
                          name='Early Warning Signal'))

fig.update_layout(
    title='Synthetic 2025 Sensex Closing Price with Early Warning Signals',
    xaxis_title='Date',
    yaxis_title='Sensex Close',
    xaxis_rangeslider_visible=True, 
    template="plotly_white"
)

fig.show()

###THATS IT